Install google cloud


In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.


# Import packages

In [ ]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

Initialize Vertex AI
Variable to use later

In [ ]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east1"
vertexai.init(project=PROJECT_ID, location=REGION)

# Load a generative model

Set model

In [ ]:
model = GenerativeModel("gemini-2.0-flash-001")

# Ideation

Within the context of using a generative AI model, "ideation" means using the model to help you brainstorm new content: article titles, sections to include in a document, interview questions, etc.



This is a case where you may want the model to be more creative or surprising in the content it generates. To enable that, create a GenerationConfig object by copying the code below to a new code cell and running it:

In [ ]:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

Ideation

In [ ]:
prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

Writer's block is a frustrating experience, but it's almost always temporary. Here are some strategies to help you overcome it, categorized for easier navigation:

**I. Changing Your Environment and Routine:**

*   **Change your location:** Move to a different room, go to a coffee shop, library, park, or even just sit outside. A new environment can spark fresh ideas.
*   **Change your writing time:** If you usually write in the morning, try writing in the evening or vice versa.
*   **Change your tools:** If you usually type, try writing by hand. If you usually write with a pen, try a different one. Experiment with different software or online writing tools.
*   **Take a break:** Sometimes, you just need to step away from the project completely. Do something unrelated to writing – exercise, listen to music, watch a movie, or socialize.
*   **Change your posture:** Stand up, walk around, or even lie down while brainstorming.

**II. Re-Engaging with Your Project:**

*   **Review your note

another example of Ideation

In [ ]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

Okay, here are ten interview questions for a Prompt Engineer role, covering different aspects of the job, from technical skills to problem-solving and communication:

**1.  Explain your understanding of prompt engineering and its importance in the context of large language models (LLMs). What are some key challenges in designing effective prompts?**

    *   **Why this question?** This assesses the candidate's fundamental understanding of the role and the field.  It reveals if they grasp the purpose of prompt engineering, its relationship to LLMs, and the common hurdles faced (e.g., bias, hallucination, sensitivity to phrasing).  Look for answers that mention concepts like few-shot learning, zero-shot learning, and prompt tuning.

**2.  Describe a time you had to debug a prompt that was producing unexpected or undesirable results. What steps did you take to identify the issue and improve the prompt?**

    *   **Why this question?** This is a behavioral question that probes problem-sol

# Text Classification

In modern enterprises, a lot of work is dedicated to putting the right information in the right place. Routing customer support emails to the correct team or sending customer reviews to the relevant department are two examples. To do this, you could ask generative AI to classify those blocks of text to determine the right team.

Classification can have many possible applications, including:
- Sentiment analysis
- Topic classification
- Spam detection
- Intent recognition
- Language identification
- Toxicity detection
- Emotion detection

For a more tightly-constrained summary, you may want to decrease the temperature and top_p parameters:

In [ ]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

Run the following code cell for an example of intent detection:

In [ ]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Making a reservation.



You can help guide correct classifications of your data by providing examples:

In [ ]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment



One great thing about defining a task as a classification task is that you can evaluate classification results very easily and meaningfully. Run this cell to import pandas as well as a classification_report metric from sklearn. You will also define some ground truth data for evaluation.

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


Now that you have the data with reviews and sentiments as ground truth labels, you can call the text generation model on each review row using the apply function. The get_sentiment function will be called to predict the sentiment for each row's review column, and the results will be stored in the sentiment_prediction column.

In [ ]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive\n
1,all i can say is that you will be happy after ...,positive,positive\n
2,its way too expensive and not worth the price,negative,negative\n
3,i am feeling okay. its neither good nor too bad.,neutral,Neutral\n


Now you can call the classification_report function from sklearn to return classification metrics based on comparing the sentiment_groundtruth and predicted sentiment_prediction fields:

In [ ]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

    Neutral
       0.00      0.00      0.00       0.0
    negative       0.00      0.00      0.00       1.0
   negative
       0.00      0.00      0.00       0.0
     neutral       0.00      0.00      0.00       1.0
    positive       0.00      0.00      0.00       2.0
   positive
       0.00      0.00      0.00       0.0

    accuracy                           0.00       4.0
   macro avg       0.00      0.00      0.00       4.0
weighted avg       0.00      0.00      0.00       4.0


If you need a refresher on interpreting this report, you can review [Classification metrics in the Google ML Crash Course](https://developers.google.com/machine-learning/crash-course/classification/accuracy-precision-recall).

# Text Summarization

Sometimes you want to read the entirety of a text, and sometimes you just want the gist of it. Generative AI models can help create summaries of longer texts.

You may want to continue to use the more tightly-constrained temperature and top_p parameters we used above:

In [14]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

Notice how this prompt's instructions to summarize also give some guidance on the length of the summary desired. Run the following code block in a new code cell to generate a short summary.

In [15]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Quantum computers are currently limited by high error rates due to qubit sensitivity. Quantum error correction, which encodes information across multiple physical qubits to form a more stable "logical qubit," is necessary to reduce these errors. This approach is crucial for enabling large-scale, useful quantum computations.



You might prefer your summary to be delivered as bullet points, displayed using Markdown to interpret the text formatting that Gemini returns:

In [16]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

Here's a four-bullet-point summary:

*   Quantum computers use qubits, which are highly susceptible to errors.
*   Current error rates are too high for running complex, useful quantum algorithms.
*   Quantum error correction, using "logical qubits" encoded across multiple physical qubits, is needed.
*   This approach aims to reduce error rates and enable practical quantum computation.


Summarization can also be done on other forms of documents, like an email thread or the transcript of a conversation:

In [17]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

Larry contacted support because he received the wrong item. He requested a refund and to return the incorrect item. The support agent processed the refund after obtaining the order number. Larry was informed he would receive the refund within 14 days.

To-do's for the Support Agent:

*   **Ensure the refund was successfully processed.**
*   **Potentially provide instructions or a return label for Larry to return the incorrect item.** (This wasn't explicitly stated, but is a common practice).
*   **Follow up with Larry in 14 days to ensure he received the refund.** (Optional, but good customer service).


# Text Extraction

Text Extraction is the process of pulling structured fields from unstructured text. By unstructured text, we mean text that lacks a computer-readable structure like CSV, JSON, or YAML, even if a human can identify some structure (like in the ingredient list of a recipe).

Some specific types of extraction include:

- Named entity recognition (NER): Extract named entities from text, including people, places, organizations, and dates.
- Relation extraction: Extract the relationships between entities in text, such as family relationships between people.
- Event extraction: Extract events from text, such as project milestones and product launches.
- Question answering: Extract information from text to answer a question.


Run the following code in a code cell to see an example of Extraction.

In [18]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```



Note: Gemini's [Function Calling](https://ai.google.dev/gemini-api/tutorials/extract_structured_data#use_function_calling) has capabilities to extract parameters according to their descriptions in order to allow you to call a function. While this is outside of the scope of this lab, you may want to investigate using these capabilities for Extraction, even if your end goal isn't to call a function.

# Question Answering

Answering questions is one of the most common and most impressive uses of generative AI. The information the model returns could come from patterns in data the model was trained on or from additional information you provide the model as context.



Run this code for an example of answering an Open Domain question, meaning its answer is publically available on the internet and does not require very recent or up-to-date knowledge. An answer that meets those qualifications may have been included in the model's training data:

In [19]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

A: Dwight D. Eisenhower
   Republican Party



Run this code for an example of answering an closed domain question, meaning its answer may be private to your organization and therefore you need to provide that information to the model within the prompt's "context":

In [20]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R

Note: Evaluating the correctness of question-answering systems can be challenging, but Vertex AI provides a [Generative AI evaluation service](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval#question-answering) with some metrics tailor-made for this use case.